In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [31]:
# parameters
tensorboard_dir = '../tb/run10';
checkPointPath = tensorboard_dir + '/best_weights.hdf5';
gpuName = '/device:GPU:1'
tensorboardFlag = True;
workers = 10;
histogram_freq = 0;

loadWeights = False;
dontFreezeLast = 5;

batchSize = 64;
epochs = 100;
validation_size=0.3;

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
df_train = pd.read_csv('../input/labels.csv')
df_test = pd.read_csv('../input/sample_submission.csv')

print('Training images: ',df_train.shape[0])
print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

Training images:  10222
Test images:  10357


In [4]:
df_train.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
im_size = 90

In [8]:
x_train = []
y_train = []
x_test = []

In [9]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('../input/train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

100%|██████████| 10222/10222 [00:49<00:00, 205.51it/s]


In [10]:
# for f in tqdm(df_test['id'].values):
#     img = cv2.imread('../input/test/{}.jpg'.format(f))
#     x_test.append(cv2.resize(img, (im_size, im_size)))

100%|██████████| 10357/10357 [00:44<00:00, 230.83it/s]


In [11]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

(10222, 90, 90, 3)
(10222, 120)
(10357, 90, 90, 3)


We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
num_class = y_train_raw.shape[1]
print('Number of classes: ', num_class)

Number of classes:  120


It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [32]:
# Create the base pre-trained model
# Can't download weights in the kernel
with tf.device(gpuName):
    base_model = VGG19(#weights='imagenet',
        weights = 'imagenet', include_top=False, input_shape=(im_size, im_size, 3))

    # Add a new top layer
    x = base_model.output
    x = Flatten()(x)
    predictions = Dense(num_class, activation='softmax')(x)

    # This is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # First: train only the top layers (which were randomly initialized)
    for i in range(len(base_model.layers)-dontFreezeLast):
        base_model.layers[i].trainable = False

    if loadWeights:
        model.load_weights(checkPointPath)

    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])

    callbacks_list = [];
    callbacks_list.append(keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=10,
        verbose=1));
    callbacks_list.append(keras.callbacks.ModelCheckpoint(
        checkPointPath,
        monitor='val_acc',
        verbose=1,
        save_best_only=True,
        mode='max',
        save_weights_only=True))
    if tensorboardFlag:
        callbacks_list.append(keras.callbacks.TensorBoard(
                log_dir=tensorboard_dir,
                histogram_freq=histogram_freq,
                write_graph=False,
                write_images=False));
        print('Tensorboard activated in directory: ',tensorboard_dir)
    else:
        print('Tensorboard NOT activated')


    model.summary()

Tensorboard activated in directory:  ../tb/run9
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 90, 90, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 90, 90, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 90, 90, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 45, 45, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 45, 45, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 45, 45, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)  

In [16]:
def generator(X,Y,batch_size):
    batch_features = np.ndarray(shape=(batch_size,) + X.shape[1:],
                                dtype=X.dtype);
    batch_labels = np.ndarray(shape=(batch_size,) + Y.shape[1:],
                                dtype=Y.dtype)
    N = X.shape[0];
    
    while True:
        for i in range(batch_size):
            # choose random index in features
            index= np.random.choice(N,1)
            batch_features[i] = X[index]
            batch_labels[i] = Y[index]
        yield batch_features, batch_labels

In [33]:
model.fit(X_train, Y_train,
          epochs=epochs,
          batch_size = batchSize,
          validation_data=(X_valid, Y_valid),
          verbose=1,
          callbacks=callbacks_list)

# steps_per_epoch = round(X_train.shape[0]/batchSize)
# model.fit_generator(generator(X_train,Y_train,batchSize),
#                    steps_per_epoch=steps_per_epoch,
#                    epochs=epochs,
#                    verbose=1,
#                    callbacks=callbacks_list,
#                    validation_data=(X_valid,Y_valid),
#                    workers=workers,
#                    use_multiprocessing=True)

Train on 7155 samples, validate on 3067 samples
Epoch 1/100
7155/7155 [==============================] - 33s 5ms/step - loss: 4.8194 - acc: 0.0078 - val_loss: 4.7864 - val_acc: 0.0104
Epoch 2/100
7155/7155 [==============================] - 31s 4ms/step - loss: 4.7856 - acc: 0.0098 - val_loss: 4.7855 - val_acc: 0.0137
Epoch 3/100
7155/7155 [==============================] - 30s 4ms/step - loss: 4.7842 - acc: 0.0109 - val_loss: 4.7848 - val_acc: 0.0137
Epoch 4/100
7155/7155 [==============================] - 30s 4ms/step - loss: 4.7830 - acc: 0.0109 - val_loss: 4.7843 - val_acc: 0.0137
Epoch 5/100
7155/7155 [==============================] - 30s 4ms/step - loss: 4.7820 - acc: 0.0117 - val_loss: 4.7839 - val_acc: 0.0137
Epoch 6/100
7155/7155 [==============================] - 30s 4ms/step - loss: 4.7811 - acc: 0.0117 - val_loss: 4.7835 - val_acc: 0.0137
Epoch 7/100
7155/7155 [==============================] - 30s 4ms/step - loss: 4.7803 - acc: 0.0117 - val_loss: 4.7833 - val_acc: 0.0137


Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [18]:
# preds = model.predict(x_test, verbose=1)

10357/10357 [==============================] - 29s 3ms/step


In [19]:
# sub = pd.DataFrame(preds)
# # Set column names to those generated by the one-hot encoding earlier
# col_names = one_hot.columns.values
# sub.columns = col_names
# # Insert the column id from the sample_submission at the start of the data frame
# sub.insert(0, 'id', df_test['id'])
# sub.head(10)

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,4.813150e-06,2.922478e-03,1.453026e-06,2.320287e-06,0.001004,2.387799e-05,0.000023,0.000201,2.309165e-03,...,3.145391e-04,8.169439e-05,1.116560e-04,2.228787e-04,1.443164e-02,2.490869e-01,2.473228e-07,2.935752e-04,0.000069,0.000003
1,00102ee9d8eb90812350685311fe5890,2.092782e-04,2.185443e-03,6.593299e-05,6.658657e-05,0.000521,1.201112e-04,0.000317,0.000082,1.404511e-04,...,8.277562e-03,1.564754e-02,1.154388e-04,2.061904e-05,9.917021e-04,6.523633e-04,2.048586e-03,4.547949e-04,0.012780,0.000149
2,0012a730dfa437f5f3613fb75efcd4ce,8.140001e-04,1.987378e-03,6.120207e-04,3.565911e-02,0.003319,5.403623e-02,0.000546,0.024864,6.407145e-03,...,3.271573e-03,2.992515e-03,2.583072e-03,2.099369e-03,1.217092e-02,4.846604e-03,3.613795e-03,5.382324e-02,0.023214,0.000308
3,001510bc8570bbeee98c8d80c8a95ec1,5.938213e-03,8.623795e-03,5.881127e-04,4.488248e-02,0.002938,2.812006e-02,0.003587,0.158284,5.207770e-05,...,4.605414e-04,3.712250e-04,1.083203e-03,1.621159e-05,2.125527e-04,4.683340e-04,1.937738e-05,1.367112e-02,0.000299,0.000463
4,001a5f3114548acdefa3d4da05474c2e,1.077847e-02,6.572982e-03,6.802641e-03,5.746588e-02,0.005621,1.296310e-03,0.007112,0.000313,4.818450e-04,...,7.303822e-04,3.645689e-04,7.969447e-04,4.750578e-05,5.069260e-03,5.581238e-05,3.447961e-04,2.752959e-03,0.005772,0.000511
5,00225dcd3e4d2410dd53239f95c0352f,2.466428e-05,3.753828e-02,3.377193e-05,8.315711e-04,0.001181,2.512980e-06,0.000715,0.001816,3.010497e-03,...,4.220240e-03,1.324339e-03,4.500544e-03,2.996731e-03,1.828388e-03,9.033299e-03,1.833412e-04,3.416817e-02,0.001677,0.000178
6,002c2a3117c2193b4d26400ce431eebd,6.011969e-03,1.477900e-02,4.874186e-03,3.338965e-02,0.002702,8.168455e-03,0.096990,0.004992,1.952365e-04,...,1.318704e-03,1.270146e-02,3.278943e-03,2.117964e-03,7.217019e-04,2.122747e-02,6.443932e-03,2.334465e-04,0.006727,0.023265
7,002c58d413a521ae8d1a5daeb35fc803,6.172440e-09,1.560974e-08,3.351846e-10,1.895891e-07,0.000044,2.127304e-07,0.000307,0.000006,3.233310e-10,...,7.692228e-08,5.150074e-08,1.002190e-11,1.828129e-10,2.500168e-13,1.619442e-07,1.173157e-01,9.709800e-08,0.033795,0.000003
8,002f80396f1e3db687c5932d7978b196,1.880752e-03,2.655190e-03,4.138285e-02,1.489929e-02,0.004972,6.552123e-03,0.004380,0.005482,6.346779e-04,...,6.282727e-04,1.611013e-04,1.061203e-03,2.197665e-04,1.196979e-02,2.115794e-03,6.495166e-04,1.171076e-02,0.003061,0.000326
9,0036c6bcec6031be9e62a257b1c3c442,1.472548e-03,5.221819e-04,6.907773e-04,4.381935e-04,0.000575,2.325841e-03,0.000446,0.000042,4.544388e-04,...,5.471634e-05,2.432812e-05,1.830640e-03,5.973030e-04,2.966550e-01,6.406604e-04,8.655368e-06,1.086839e-04,0.000334,0.000098
